First we need to import all of our data.

In [150]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier

# Read in the data
conditionData = []
controlData = []
scores = pd.read_csv("data/scores.csv")

for fileName in os.listdir("data/condition"):
	conditionData.append(pd.read_csv(f"data/condition/{fileName}"))
for fileName in os.listdir(f"data/control"):
	controlData.append(pd.read_csv(f"data/control/{fileName}"))

Now we need to combine our motor activity in one file with the labels indicating if it is a condition or control

In [151]:
df = pd.DataFrame(columns=["Activity", "Target"])
for data in conditionData:
	activity = data.activity
	activity = np.array(activity)
	temp = {'Activity': activity, 'Target': "condition"}
	dfTemp = pd.DataFrame(temp)
	df = pd.concat([df, dfTemp])

for data in controlData:
	activity = data.activity
	activity = np.array(activity)
	temp = {'Activity': activity, 'Target': "control"}
	dfTemp = pd.DataFrame(temp)
	df = pd.concat([df, dfTemp])


Split our data into training/testing

In [156]:
y = df.iloc[:,-1:]
x = df.iloc[: , :-1]

y = np.array(y)
x = np.array(x)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3)
sss.get_n_splits(x, y)
for train_index, test_index in sss.split(x, y):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]


Use a Random Forest Classifier.

In [157]:
clf=RandomForestClassifier(n_estimators=10)
clf.fit(X_train, y_train.ravel())
accuracy = clf.score(X_test, y_test)
print(accuracy)

0.6635250004241674


Our classifier isn't very good. Hits around 66% with a .3 split. We might want to clean the data a bit more or use a different classifier.